In [47]:
import pandas as pd
import numpy as np
import math
from pulp import *
from tabulate import tabulate

In [241]:
# ========= SET-UP =========== #
NUM_AREA = 3
R = 9.03 # Revenue
S = 1 # Multiplier

# ========== DATA ============= #

x_ij = [[0.99, 0.68, 0.17], 
        [0.68, 0.962, 0.43], 
        [0.17,0.44,0.99]] # [[row]] - x00, x01, x10, x11, move cars from j to i

c_ij = [[0, 4.14, 6.52], 
        [4.14, 0, 5.18], 
        [6.52,5.18,0]] # c00, c01, c10, c11 # Move cars from i to j


d_i = [100,20,0]
n_0i = [5,44,9999]

# ========= INIT LP ============ #
prob = LpProblem("Model", LpMaximize)

m_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
y_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
n_1i = []
for i in range(NUM_AREA):
    n_1i.append(LpVariable("n_1%d"%(i), 0, None, LpInteger))
    for j in range(NUM_AREA):
        y_ij[i][j] = LpVariable("y_%d%d"%(i,j), 0, None, LpInteger)
        if i!=j:
            m_ij[i][j] = LpVariable("m_%d%d"%(i,j), 0, None, LpInteger)

# ========= CONSTRUCT LP ============ #

# Objective
prob += sum([S*R*y_ij[i][j]*x_ij[i][j] - m_ij[i][j]*c_ij[i][j] for i in range(NUM_AREA) for j in range(NUM_AREA)])

# Subject To
for j in range(NUM_AREA):
    # n_1i
    prob += n_1i[j] == n_0i[j] + sum(m_ij[i][j] for i in range(NUM_AREA)) - sum(m_ij[j][i] for i in range(NUM_AREA)), "Equation Area: %d"%j
    # n_0i
    prob += n_0i[j] >= sum(m_ij[j][i] for i in range(NUM_AREA)), "Moving Constrain: %d"%j
    # y_ij
    prob += sum(y_ij[j][i] for i in range(NUM_AREA)) <= d_i[j] ,"Total Demand: %d"%j
    prob += sum(y_ij[i][j]*x_ij[i][j] for i in range(NUM_AREA)) <= n_1i[j] ,"Total Available: %d"%j
    

# ============== SOLVE ============= #
prob.solve()
print("\n ======== Optimal Variables ========= \n")
for v in prob.variables():
    print (v.name, "=", v.varValue)
print("Objective: " ,value(prob.objective))

print("\n ======== Model Details ========= \n")
results = []
for name, c in list(prob.constraints.items()):
    results.append([name,c,  c.pi,  c.slack])
print (tabulate(results, headers=['Constrain', 'Formula', "pi", "slack"]))


 ======== Optimal Variables ========= 

m_01 = 0.0
m_02 = 0.0
m_10 = 0.0
m_12 = 0.0
m_20 = 1.0
m_21 = 20.0
n_10 = 6.0
n_11 = 64.0
n_12 = 9978.0
y_00 = 6.0
y_01 = 94.0
y_02 = 0.0
y_10 = 0.0
y_11 = 0.0
y_12 = 20.0
y_20 = 0.0
y_21 = 0.0
y_22 = 0.0
Objective:  598.3738

 ======== Model Details ========= 

Constrain            Formula                                               pi    slack
-------------------  --------------------------------------------------  ----  -------
Equation_Area:_0     m_01 + m_02 - m_10 - m_20 + n_10 = 5                  -0    -0
Moving_Constrain:_0  m_01 + m_02 <= 5                                      -0     5
Total_Demand:_0      y_00 + y_01 + y_02 <= 100                             -0    -0
Total_Available:_0   -n_10 + 0.99*y_00 + 0.68*y_10 + 0.17*y_20 <= -0.0     -0     0.06
Equation_Area:_1     -m_01 + m_10 + m_12 - m_21 + n_11 = 44                -0    -0
Moving_Constrain:_1  m_10 + m_12 <= 44                                     -0    44
Total_Demand:_1

In [225]:
def optimizer (x_ij, c_ij, d_i, n_0i, R, NUM_AREA):
    prob = LpProblem("Model", LpMaximize)

    m_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
    y_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
    n_1i = []
    for i in range(NUM_AREA):
        n_1i.append(LpVariable("n_1%d"%(i), 0, None, LpInteger))
        for j in range(NUM_AREA):
            y_ij[i][j] = LpVariable("y_%d%d"%(i,j), 0, None, LpInteger)
            if i!=j:
                m_ij[i][j] = LpVariable("m_%d%d"%(i,j), 0, None, LpInteger)

    # ========= CONSTRUCT LP ============ #

    # Objective
    prob += sum([R*y_ij[i][j]*x_ij[i][j] - m_ij[i][j]*c_ij[i][j] for i in range(NUM_AREA) for j in range(NUM_AREA)])

    # Subject To
    for j in range(NUM_AREA):
        # n_1i
        prob += n_1i[j] == n_0i[j] + sum(m_ij[i][j] for i in range(NUM_AREA)) - sum(m_ij[j][i] for i in range(NUM_AREA)), "Equation Area: %d"%j
        # n_0i
        prob += n_0i[j] >= sum(m_ij[j][i] for i in range(NUM_AREA)), "Moving Constrain: %d"%j
        # y_ij
        prob += sum(y_ij[j][i] for i in range(NUM_AREA)) <= d_i[j] ,"Total Demand: %d"%j
        prob += sum(y_ij[i][j]*x_ij[i][j] for i in range(NUM_AREA)) <= n_1i[j] ,"Total Available: %d"%j


    # ============== SOLVE ============= #
    prob.solve()
    '''
    print("\n ======== Optimal Variables ========= \n")
    
    for v in prob.variables():
        print (v.name, "=", v.varValue)
    print("Objective: " ,value(prob.objective))

    print("\n ======== Model Details ========= \n")
    results = []
    for name, c in list(prob.constraints.items()):
        results.append([name,c,  c.pi,  c.slack])
    print (tabulate(results, headers=['Constrain', 'Formula', "pi", "slack"]))
    '''
    return prob.variables()[6:8]

In [227]:
# ========= SET-UP =========== #
NUM_AREA = 3
R = 11.5 # Revenue
S = 1.6 # Multiplier

# ========== DATA ============= #

# X_ij is very sensitive
x_ij = [[.985, 0.7, 0.2], 
        [0.7, .985, 0.2], 
        [0,0,0]] # [[row]] - x00, x01, x10, x11, move cars from j to i

c_ij = [[0, 4.5, 999], 
        [4.5, 0, 999], 
        [6,6,0]] # c00, c01, c10, c11 # Move cars from i to j

d_i = [100,20,0]
n_0i = [10,38,9999]

res  = optimizer (x_ij, c_ij, d_i, n_0i, R*S, NUM_AREA)
res

[n_10, n_11]

In [242]:
NUM_AREA = 3
R = 9.03 # Revenue
S = 1 # Multiplier

# ========== DATA ============= #

x_ij = [[0.99, 0.68, 0.17], 
        [0.68, 0.962, 0.43], 
        [0.17,0.44,0.99]] # [[row]] - x00, x01, x10, x11, move cars from j to i

c_ij = [[0, 4.14, 6.52], 
        [4.14, 0, 5.18], 
        [6.52,5.18,0]] # c00, c01, c10, c11 # Move cars from i to j

n_0i = [5,44,9999]

res = pd.DataFrame(columns = [ "n_10", "n_11"])

# out = optimizer (x_ij, c_ij, [100,20,0], n_0i, R*S, NUM_AREA)

for d1 in range(80,200):
    d_i = [d1, 20,0]
    out = optimizer (x_ij, c_ij, d_i, n_0i, R*S, NUM_AREA)
    
    res.loc[d1] = [out[0].varValue, out[1].varValue]

res


,n_10,n_11
80,5.0,51.0
81,6.0,51.0
82,7.0,51.0
83,5.0,55.0
84,6.0,55.0
85,7.0,55.0
86,5.0,58.0
87,6.0,58.0
88,7.0,58.0
89,5.0,61.0


In [240]:
res.to_csv("loop_di.csv")